In [ ]:
### Packages
import os
import sys
import wandb
import numpy as np
import torch
from argparse import ArgumentParser

from wandb.sdk import wandb_run

import time
sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
sys.path.append('/home/augustsemrau/drive/bachelor/TGML/src')



### Code imports
## Data
from data.synthetic.datasets.init_params import get_initial_parameters
from data.synthetic.builder import DatasetBuilder
from data.synthetic.stepwisebuilder import StepwiseDatasetBuilder
from data.synthetic.sampling.constantvelocity import ConstantVelocitySimulator
from data.synthetic.sampling.tensor_stepwiseconstantvelocity import StepwiseConstantVelocitySimulator

## Models
from models.nodynamics import NoDynamicsModel
from models.constantvelocity.standard import ConstantVelocityModel
from models.constantvelocity.vectorized import VectorizedConstantVelocityModel
from models.constantvelocity.stepwise import StepwiseVectorizedConstantVelocityModel
from models.constantvelocity.stepwise_stepbeta import StepwiseVectorizedConstantVelocityModel as MultiBetaStepwise
from models.constantvelocity.standard_gt import GTConstantVelocityModel  
from models.constantvelocity.stepwise_gt import GTStepwiseConstantVelocityModel
from models.constantvelocity.stepwise_gt_stepbeta import GTStepwiseConstantVelocityModel as GTMultiBetaStepwise

## Training Gym's
from traintestgyms.ignitegym import TrainTestGym


import matplotlib.pyplot as plt

In [ ]:
### Parse Arguments for running in terminal
    arg_parser = ArgumentParser()
    arg_parser.add_argument('--seed', '-seed', default=1, type=int)
    arg_parser.add_argument('--device', '-device', default='cpu', type=str)
    arg_parser.add_argument('--learning_rate', '-LR', default=0.025, type=float)
    arg_parser.add_argument('--num_epochs', '-NE', default=10, type=int)
    arg_parser.add_argument('--train_batch_size', '-TBS', default=-1, type=int)
    arg_parser.add_argument('--real_data', '-RD', default=0, type=int)
    arg_parser.add_argument('--dataset_number', '-DS', default=3, type=int)
    arg_parser.add_argument('--training_type', '-TT', default=0, type=int)
    arg_parser.add_argument('--vectorized', '-VEC', default=2, type=int)
    arg_parser.add_argument('--remove_node_pairs_b', '-T1', default=0, type=int)
    arg_parser.add_argument('--remove_interactions_b', '-T2', default=0, type=int)
    arg_parser.add_argument('--steps', '-steps', default=0, type=int)
    arg_parser.add_argument('--step_beta', '-SB', action='store_true')
    arg_parser.add_argument('--keep_rotation', '-KR', action='store_true')
    arg_parser.add_argument('--animation', '-ani', action='store_true')
    arg_parser.add_argument('--animation_time_points', '-ATP', default=500, type=int)
    arg_parser.add_argument('--velocity_gamma_regularization', '-VGR', default=None, type=float)
    arg_parser.add_argument('--wandb_entity', '-WE', default='augustsemrau', type=str)
    arg_parser.add_argument('--wandb_project', '-WP', default='TGMLRQ2_1', type=str)
    arg_parser.add_argument('--wandb_run_name', '-WRN', default=None, type=str)
    arg_parser.add_argument('--wandb_group', '-WG', default=None, type=str)
    args = arg_parser.parse_args()

    ## Set all input arguments
    seed = args.seed
    learning_rate = args.learning_rate
    num_epochs = args.num_epochs
    train_batch_size = args.train_batch_size
    dataset_number = args.dataset_number
    training_type = args.training_type
    vectorized = args.vectorized
    remove_node_pairs_b = args.remove_node_pairs_b
    remove_interactions_b = args.remove_interactions_b
    device = args.device
    real_data = args.real_data
    num_steps = args.steps
    step_beta = args.step_beta
    keep_rotation = args.keep_rotation
    animation = args.animation
    animation_time_points = args.animation_time_points
    velocity_gamma_regularization = args.velocity_gamma_regularization
    wandb_entity= args.wandb_entity
    wandb_project = args.wandb_project
    wandb_run_name = args.wandb_run_name
    wandb_group = args.wandb_group

    ## Seeding of model run
    np.random.seed(seed)
    torch.manual_seed(seed)
    np.seterr(all='raise')

 
    ### WandB initialization
    ## Set input parameters as config for Weights and Biases
    wandb_config = {'seed': seed,
                    'device': device,
                    'learning_rate': learning_rate,
                    'vectorized': vectorized,  # 0 = non-vectorized, 1 = vectorized, 2 = stepwise
                    'training_type': training_type,  # 0 = non-sequential training, 1 = sequential training
                    'num_epochs': num_epochs,
                    'remove_nodepairs': remove_node_pairs_b,
                    'remove_interactions': remove_interactions_b,
                    'num_steps': num_steps,
                    'train_batch_size': train_batch_size,
                    'velocity_gamma_regularization': velocity_gamma_regularization
                    }

# Testing node count


In [ ]:
## Initialize WandB for logging config and metrics
wandb.init(project=wandb_project, name=wandb_run_name, 
            entity=wandb_entity, group=wandb_group, config=wandb_config)


## Device
for dev in ['cpu','cuda']:
    device = dev
    print(f'Running with pytorch device: {device}')
    torch.pi = torch.tensor(torch.acos(torch.zeros(1)).item()*2).to(device)
    torch.eps = torch.tensor(np.finfo(float).eps).to(device) 
    
    mean_runtimes = []

    interaction_count = []
    num_nodes_list = [2, 4 ,6 ,8 ,10,12,14,16,18,20,22 ,24 ,26 ,28, 30 ,34 ,38, 42, 46, 50, 60]

    for i in num_nodes_list:
        num_nodes = i
        num_steps = 4
        beta = 5.
        runtimes = []
        for i in range(1):

            z0, v0, true_beta, model_beta, max_time = get_initial_parameters(dataset_number=dataset_number, vectorized=vectorized, num_nodes=num_nodes, beta=beta)
            if num_steps == 0:
                num_steps = v0.shape[2]
            num_nodes = z0.shape[0]

            simulator = StepwiseConstantVelocitySimulator(starting_positions=z0, velocities=v0, max_time=max_time, beta=true_beta, seed=seed)
            data_builder = StepwiseDatasetBuilder(simulator=simulator, device=device, normalization_max_time=None)
            dataset_full = data_builder.build_dataset(num_nodes, time_column_idx=2)
            dataset = dataset_full[:10000]
            dataset = dataset_full
            max_time = max(dataset[:,2])

            dataset_size = len(dataset)
            print(dataset_size)
            if i == 0 and dev == 'cuda':
                interaction_count.append(dataset_size)

            num_dyads = (num_nodes * (num_nodes - 1)) / 2
            train_batch_size = train_batch_size if train_batch_size > 0 else dataset_size

            
            ### Setup Model: Either non-vectorized, vectorized or stepwise
            if vectorized == -1:
                model = NoDynamicsModel(n_points=num_nodes, beta=model_beta).to(device, dtype=torch.float32)
            if vectorized == 0:
                model = ConstantVelocityModel(n_points=num_nodes, beta=model_beta).to(device, dtype=torch.float32)
            elif vectorized == 1:
                model = VectorizedConstantVelocityModel(n_points=num_nodes, beta=model_beta, device=device, z0=z0, v0=v0, true_init=True).to(device, dtype=torch.float32)
            elif vectorized == 2:
                last_time_point = dataset[:,2][-1].item()
                if isinstance(model_beta, np.ndarray):
                    model = MultiBetaStepwise(n_points=num_nodes, beta=model_beta, steps=num_steps, max_time=last_time_point, 
                                                device=device, z0=z0, v0=v0, true_init=False).to(device, dtype=torch.float32)
                else:
                    model = StepwiseVectorizedConstantVelocityModel(n_points=num_nodes, beta=model_beta, steps=num_steps, 
                                    max_time=last_time_point, device=device, z0=z0, v0=v0, v0_init=training_type, 
                                    gamma=velocity_gamma_regularization).to(device, dtype=torch.float32)
                    
            ## Optimizer is initialized here, Adam is used
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

            ### Model training: Either non-sequential or sequential
            metrics = {'avg_train_loss': [], 'beta_est': []}
            
            ## Non-sequential model training
            if training_type == 0:
                model.z0.requires_grad, model.v0.requires_grad, model.beta.requires_grad = True, True, True
                gym = TrainTestGym(dataset=dataset, 
                                    model=model, 
                                    device=device, 
                                    batch_size=train_batch_size, 
                                    optimizer=optimizer, 
                                    metrics=metrics, 
                                    time_column_idx=2,
                                    wandb_handler = wandb,
                                    num_dyads=num_dyads,
                                    keep_rotation=keep_rotation)
                start_time = time.time()
                gym.train_test_model(epochs=num_epochs)
                end_time = time.time()
            
            runtime = (end_time - start_time)
            runtimes.append(runtime)
            print('')
            print(runtime)
            print(num_nodes)
            print(num_steps)
            print(beta)
        mean_runtimes.append(sum(runtimes) / 5)
        # print(mean_runtimes)
    if dev == 'cpu':
        cpu_times = mean_runtimes
        wandb.log({'cpu_times': cpu_times})
    else:
        cuda_times = mean_runtimes
        wandb.log({'cuda_times': cuda_times})
        
print(interaction_count)
print(cpu_times)
print(cuda_times)

fig, ax = plt.subplots(1,1, figsize=(10, 6), facecolor='w', edgecolor='k')
ax.plot(num_nodes_list, np.asarray(cpu_times)/num_epochs, linestyle='-', label='CPU')
ax.plot(num_nodes_list, np.asarray(cuda_times)/num_epochs, linestyle='-', label='CUDA')
ax.legend()
ax.set_xlabel('Number of Nodes')
ax.set_ylabel('Runtime per Epoch in Seconds')
ax.set_title('Runtime per Epoch given number of Nodes for SCVM on CPU and CUDA')

wandb.log({'Interactions_runtime_Plot': wandb.Image(fig)})
plt.show()

# Testing step count


In [ ]:
## Device
for dev in ['cpu','cuda']:
    device = dev
    print(f'Running with pytorch device: {device}')
    torch.pi = torch.tensor(torch.acos(torch.zeros(1)).item()*2).to(device)
    torch.eps = torch.tensor(np.finfo(float).eps).to(device) 
    mean_runtimes = []
    interaction_count = []

    num_steps_list = [4,8,12,16,20,30,40,50,75,100,150,200,250,300,350,400,450,500,750,1000]
    for i in num_steps_list:
        num_nodes = 4
        num_steps = i
        beta = 5.
        runtimes = []
        for i in range(1):

            z0, v0, true_beta, model_beta, max_time = get_initial_parameters(dataset_number=dataset_number, vectorized=vectorized, num_nodes=num_nodes, beta=beta)
            if num_steps == 0:
                num_steps = v0.shape[2]
            num_nodes = z0.shape[0]

            simulator = StepwiseConstantVelocitySimulator(starting_positions=z0, velocities=v0, max_time=max_time, beta=true_beta, seed=seed)
            data_builder = StepwiseDatasetBuilder(simulator=simulator, device=device, normalization_max_time=None)
            dataset_full = data_builder.build_dataset(num_nodes, time_column_idx=2)
            # dataset = dataset_full[:10000]
            dataset = dataset_full
            max_time = max(dataset[:,2])

            dataset_size = len(dataset)
            print(dataset_size)
            if i == 0 and dev == 'cuda':
                interaction_count.append(dataset_size)

            num_dyads = (num_nodes * (num_nodes - 1)) / 2
            train_batch_size = train_batch_size if train_batch_size > 0 else dataset_size

            
            ### Setup Model: Either non-vectorized, vectorized or stepwise
            if vectorized == -1:
                model = NoDynamicsModel(n_points=num_nodes, beta=model_beta).to(device, dtype=torch.float32)
            if vectorized == 0:
                model = ConstantVelocityModel(n_points=num_nodes, beta=model_beta).to(device, dtype=torch.float32)
            elif vectorized == 1:
                model = VectorizedConstantVelocityModel(n_points=num_nodes, beta=model_beta, device=device, z0=z0, v0=v0, true_init=True).to(device, dtype=torch.float32)
            elif vectorized == 2:
                last_time_point = dataset[:,2][-1].item()
                if isinstance(model_beta, np.ndarray):
                    model = MultiBetaStepwise(n_points=num_nodes, beta=model_beta, steps=num_steps, max_time=last_time_point, 
                                                device=device, z0=z0, v0=v0, true_init=False).to(device, dtype=torch.float32)
                else:
                    model = StepwiseVectorizedConstantVelocityModel(n_points=num_nodes, beta=model_beta, steps=num_steps, 
                                    max_time=last_time_point, device=device, z0=z0, v0=v0, v0_init=training_type, 
                                    gamma=velocity_gamma_regularization).to(device, dtype=torch.float32)
                    
            ## Optimizer is initialized here, Adam is used
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

            ### Model training: Either non-sequential or sequential
            metrics = {'avg_train_loss': [], 'beta_est': []}
            
            ## Non-sequential model training
            if training_type == 0:
                model.z0.requires_grad, model.v0.requires_grad, model.beta.requires_grad = True, True, True
                gym = TrainTestGym(dataset=dataset, 
                                    model=model, 
                                    device=device, 
                                    batch_size=train_batch_size, 
                                    optimizer=optimizer, 
                                    metrics=metrics, 
                                    time_column_idx=2,
                                    wandb_handler = wandb,
                                    num_dyads=num_dyads,
                                    keep_rotation=keep_rotation)
                start_time = time.time()
                gym.train_test_model(epochs=num_epochs)
                end_time = time.time()
            
            runtime = (end_time - start_time)
            runtimes.append(runtime)
            print('')
            print(runtime)
            print(num_nodes)
            print(num_steps)
            print(beta)
        mean_runtimes.append(sum(runtimes) / 5)
        # print(mean_runtimes)
    if dev == 'cpu':
        cpu_times = mean_runtimes
        wandb.log({'cpu_times': cpu_times})

    else:
        cuda_times = mean_runtimes
        wandb.log({'cuda_times': cuda_times})
        
print(interaction_count)
print(cpu_times)
print(cuda_times)
fig, ax = plt.subplots(1,1, figsize=(10, 6), facecolor='w', edgecolor='k')
ax.plot(num_steps_list, np.asarray(cpu_times)/num_epochs, linestyle='-', label='CPU')
ax.plot(num_steps_list, np.asarray(cuda_times)/num_epochs, linestyle='-', label='CUDA')
ax.legend()
ax.set_xlabel('Number of Steps')
ax.set_ylabel('Runtime per Epoch in Seconds')
ax.set_title('Runtime per Epoch given number of Steps for SCVM on CPU and CUDA')

wandb.log({'Steps_runtime_Plot': wandb.Image(fig)})
plt.show()

In [ ]:
## Device
for dev in ['cpu','cuda']:
    device = dev
    print(f'Running with pytorch device: {device}')
    torch.pi = torch.tensor(torch.acos(torch.zeros(1)).item()*2).to(device)
    torch.eps = torch.tensor(np.finfo(float).eps).to(device) 
    mean_runtimes = []

    interaction_count = []
    beta_list = [5.0, 5.25, 5.5, 5.75, 6.0, 6.25, 6.5, 6.75, 7.0, 7.25, 7.5, 7.75, 8.0, 8.25, 8.5, 8.75, 9.0]
    for i in beta_list:
        num_nodes = 4
        num_steps = 4
        beta = i
        runtimes = []
        for i in range(1):

            z0, v0, true_beta, model_beta, max_time = get_initial_parameters(dataset_number=dataset_number, vectorized=vectorized, num_nodes=num_nodes, beta=beta)
            if num_steps == 0:
                num_steps = v0.shape[2]
            num_nodes = z0.shape[0]

            simulator = StepwiseConstantVelocitySimulator(starting_positions=z0, velocities=v0, max_time=max_time, beta=true_beta, seed=seed)
            data_builder = StepwiseDatasetBuilder(simulator=simulator, device=device, normalization_max_time=None)
            dataset_full = data_builder.build_dataset(num_nodes, time_column_idx=2)
            # dataset = dataset_full[:10000]
            dataset = dataset_full
            max_time = max(dataset[:,2])

            dataset_size = len(dataset)
            print(dataset_size)
            if i == 0 and dev == 'cuda':
                interaction_count.append(dataset_size)

            num_dyads = (num_nodes * (num_nodes - 1)) / 2
            train_batch_size = train_batch_size if train_batch_size > 0 else dataset_size

            
            ### Setup Model: Either non-vectorized, vectorized or stepwise
            if vectorized == -1:
                model = NoDynamicsModel(n_points=num_nodes, beta=model_beta).to(device, dtype=torch.float32)
            if vectorized == 0:
                model = ConstantVelocityModel(n_points=num_nodes, beta=model_beta).to(device, dtype=torch.float32)
            elif vectorized == 1:
                model = VectorizedConstantVelocityModel(n_points=num_nodes, beta=model_beta, device=device, z0=z0, v0=v0, true_init=True).to(device, dtype=torch.float32)
            elif vectorized == 2:
                last_time_point = dataset[:,2][-1].item()
                if isinstance(model_beta, np.ndarray):
                    model = MultiBetaStepwise(n_points=num_nodes, beta=model_beta, steps=num_steps, max_time=last_time_point, 
                                                device=device, z0=z0, v0=v0, true_init=False).to(device, dtype=torch.float32)
                else:
                    model = StepwiseVectorizedConstantVelocityModel(n_points=num_nodes, beta=model_beta, steps=num_steps, 
                                    max_time=last_time_point, device=device, z0=z0, v0=v0, v0_init=training_type, 
                                    gamma=velocity_gamma_regularization).to(device, dtype=torch.float32)
                    
            ## Optimizer is initialized here, Adam is used
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

            ### Model training: Either non-sequential or sequential
            metrics = {'avg_train_loss': [], 'beta_est': []}
            
            ## Non-sequential model training
            if training_type == 0:
                model.z0.requires_grad, model.v0.requires_grad, model.beta.requires_grad = True, True, True
                gym = TrainTestGym(dataset=dataset, 
                                    model=model, 
                                    device=device, 
                                    batch_size=train_batch_size, 
                                    optimizer=optimizer, 
                                    metrics=metrics, 
                                    time_column_idx=2,
                                    wandb_handler = wandb,
                                    num_dyads=num_dyads,
                                    keep_rotation=keep_rotation)
                start_time = time.time()
                gym.train_test_model(epochs=num_epochs)
                end_time = time.time()
            
            runtime = (end_time - start_time)
            runtimes.append(runtime)
            print('')
            print(runtime)
            print(num_nodes)
            print(num_steps)
            print(beta)
        mean_runtimes.append(sum(runtimes) / 5)
        # print(mean_runtimes)
    if dev == 'cpu':
        cpu_times = mean_runtimes
        wandb.log({'cpu_times': cpu_times})

    else:
        cuda_times = mean_runtimes
        wandb.log({'cuda_times': cuda_times})
        
print(interaction_count)
print(cpu_times)
print(cuda_times)
fig, ax = plt.subplots(1,1, figsize=(10, 6), facecolor='w', edgecolor='k')
ax.plot(interaction_count, np.asarray(cpu_times)/num_epochs, linestyle='-', label='CPU')
ax.plot(interaction_count, np.asarray(cuda_times)/num_epochs, linestyle='-', label='CUDA')
ax.legend()
ax.set_xlabel('Number of Interactions')
ax.set_ylabel('Runtime per Epoch in Seconds')
ax.set_title('Runtime per Epoch given number of Interactions for SCVM on CPU and CUDA')

wandb.log({'Interactions_runtime_Plot': wandb.Image(fig)})
plt.show()